<a href="https://colab.research.google.com/github/cookiestroke/PySpark-Playground/blob/main/PySpark_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Spark

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main am

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [ ]:
!ls

drive  sample_data  spark-2.3.1-bin-hadoop2.7  spark-2.3.1-bin-hadoop2.7.tgz


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [ ]:
from pyspark.sql.functions import to_timestamp,col,lit
rc=spark.read.csv("/content/drive/MyDrive/Crimes_-_2001_to_Present.csv",header=True).withColumn('Date',to_timestamp(col('Date'),'MM/DD/YYYY hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11227508|   JB146365|2017-01-01 00:01:00|  027XX S WHIPPLE ST|1754|OFFENSE INVOLVING...|AGG SEX ASSLT OF ...|           RESIDENCE| false|   false|1033|     010|  12|            30|      

In [ ]:
rc.select('Primary Type').show(5)

+--------------------+
|        Primary Type|
+--------------------+
|OFFENSE INVOLVING...|
|  DECEPTIVE PRACTICE|
|OFFENSE INVOLVING...|
|OFFENSE INVOLVING...|
|         SEX OFFENSE|
+--------------------+
only showing top 5 rows



In [ ]:
rc.withColumn('Mega Villain', lit('Mandar')).show(5)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|Mega Villain|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+------------+
|11227508|   JB146365|2017-01-01 00:01:00|  027XX S WHIPPLE ST|1754|OFFENSE INVOLVING...|AGG SEX ASSLT OF ...|           RESIDENCE| false|   false|1

In [ ]:
rc.filter(col('Domestic')=='true').show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11227505|   JB146342|2017-01-01 00:01:00| 115XX S ABERDEEN ST|1751|OFFENSE INVOLVING...|CRIM SEX ABUSE BY...|           RESIDE

In [ ]:
rc.select('Primary Type').distinct().show()

+--------------------+
|        Primary Type|
+--------------------+
|OFFENSE INVOLVING...|
|CRIMINAL SEXUAL A...|
|            STALKING|
|PUBLIC PEACE VIOL...|
|           OBSCENITY|
|               ARSON|
|            GAMBLING|
|   CRIMINAL TRESPASS|
|             ASSAULT|
|LIQUOR LAW VIOLATION|
| MOTOR VEHICLE THEFT|
|               THEFT|
|             BATTERY|
|             ROBBERY|
|            HOMICIDE|
|    PUBLIC INDECENCY|
| CRIM SEXUAL ASSAULT|
|   HUMAN TRAFFICKING|
|        INTIMIDATION|
|        PROSTITUTION|
+--------------------+
only showing top 20 rows



In [ ]:
rc.orderBy(col('ID')).show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10000865|   HY190414|2012-01-01 08:00:00|   009XX N CROSBY ST|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|               OTHER|

In [ ]:
rc.filter(col('ID') < lit(1001000)).show()

+-----+-----------+-------------------+--------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|   ID|Case Number|               Date|               Block|IUCR|Primary Type|        Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-----+-----------+-------------------+--------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|20265|   HV141190|2012-01-01 19:30:00|     019XX S TROY ST|0110|    HOMICIDE|FIRST DEGREE MURDER|              STREET| false|   false|1022|     010|  24|         

In [ ]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
arrestsdone = rc.filter(col('Arrest')=='true').count()

9761

In [ ]:
totalsize = rc.select('Arrest').count()

41800

In [ ]:
print(f'percentage of arrests = {arrestsdone/totalsize*100}')

percentage of arrests = 23.351674641148325


In [ ]:
rc.groupby("Location Description").count().orderBy("count", ascending=False).show(3)

+--------------------+-----+
|Location Description|count|
+--------------------+-----+
|           RESIDENCE| 9681|
|              STREET| 8720|
|           APARTMENT| 5084|
+--------------------+-----+
only showing top 3 rows



In [ ]:
from pyspark.sql import functions

print(dir(functions))

['AutoBatchedSerializer', 'Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PickleSerializer', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'asc', 'ascii', 'asin', 'atan', 'atan2', 'avg', 'base64', 'bin', 'bitwiseNOT', 'blacklist', 'broadcast', 'bround', 'cbrt', 'ceil', 'coalesce', 'col', 'collect_list', 'collect_set', 'column', 'concat', 'concat_ws', 'conv', 'c

In [ ]:
from pyspark.sql.functions import lower, upper, substring

In [ ]:
rc.select(lower(col('Primary Type')), upper(col('Primary Type')), substring('Primary Type',1,4)).show(5)

+--------------------+--------------------+-----------------------------+
| lower(Primary Type)| upper(Primary Type)|substring(Primary Type, 1, 4)|
+--------------------+--------------------+-----------------------------+
|offense involving...|OFFENSE INVOLVING...|                         OFFE|
|  deceptive practice|  DECEPTIVE PRACTICE|                         DECE|
|offense involving...|OFFENSE INVOLVING...|                         OFFE|
|offense involving...|OFFENSE INVOLVING...|                         OFFE|
|         sex offense|         SEX OFFENSE|                         SEX |
+--------------------+--------------------+-----------------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import min, max, mean

In [ ]:
rc.select(min(col("Date")),max(col("Date"))).show(1)

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2006-01-01 00:00:00|2017-01-01 23:59:00|
+-------------------+-------------------+



In [ ]:
from pyspark.sql.functions import date_add,date_sub

In [ ]:
rc.select(date_add(min(col("Date")),3),date_sub(max(col("Date")),3)).show(1)

+----------------------+----------------------+
|date_add(min(Date), 3)|date_sub(max(Date), 3)|
+----------------------+----------------------+
|            2006-01-04|            2016-12-29|
+----------------------+----------------------+



In [ ]:
cols=['name','subject','age']
df = spark.createDataFrame([('mandar','history',24)],cols)
df.show()

+------+-------+---+
|  name|subject|age|
+------+-------+---+
|mandar|history| 24|
+------+-------+---+



In [ ]:
newrow=spark.createDataFrame([('mandy','science',2)],cols)

df = df.union(newrow)

df.show()

+------+-------+---+
|  name|subject|age|
+------+-------+---+
|mandar|history| 24|
| mandy|science|  2|
+------+-------+---+



In [ ]:
rc1=spark.read.csv('/content/Police_Stations.csv', header=True)
rc1.show()

+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|            DISTRICT|    DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|        Headquarters|     Headquarters| 3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|                  18|       Near North|  1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 11

In [ ]:
rc.cache()
rc.count()

41800

In [ ]:
rc1.select("DISTRICT").distinct().show(30)

+--------------------+
|            DISTRICT|
+--------------------+
|                   7|
|                  15|
|                  11|
|                   3|
|                   8|
|                  22|
|                  16|
|                   5|
|                  18|
|                  17|
|                   6|
|                  19|
|                  25|
|        Headquarters|
|                  24|
|                   9|
|                   1|
|                  20|
|                  10|
|                   4|
|                  12|
|                  14|
|                   2|
|",Chicago,IL,6060...|
+--------------------+



In [ ]:
rc.select("District").distinct().show(30)

+--------+
|District|
+--------+
|     009|
|     012|
|     024|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     005|
|     003|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
|     004|
|     017|
|     007|
|     002|
+--------+



In [ ]:
from pyspark.sql.functions import lpad

help(lpad)

Help on function lpad in module pyspark.sql.functions:

lpad(col, len, pad)
    Left-pad the string column to width `len` with `pad`.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(lpad(df.s, 6, '#').alias('s')).collect()
    [Row(s='##abcd')]
    
    .. versionadded:: 1.5



In [ ]:
rc1.select(lpad(col("DISTRICT"),3,'0')).distinct().show(30)

+--------------------+
|lpad(DISTRICT, 3, 0)|
+--------------------+
|                 009|
|                 012|
|                 024|
|                 Hea|
|                 015|
|                 006|
|                 019|
|                 020|
|                 011|
|                 025|
|                 003|
|                 005|
|                 016|
|                 018|
|                 008|
|                 022|
|                 001|
|                 014|
|                 010|
|                 004|
|                 017|
|                 ",C|
|                 007|
|                 002|
+--------------------+



In [ ]:
rc1=rc1.withColumn("Format_dist",lpad(col("DISTRICT"),3,'0'))
rc1.show(10)

+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-----------+
|    DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|Format_dist|
+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-----------+
|Headquarters|  Headquarters| 3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|        Hea|
|          18|    Near North|  1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.0

In [ ]:
rc.join(rc1, rc.District == rc1.Format_dist,'inner').show(5)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-----------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|Format_dist|
+--------+-----------+------------------

In [ ]:
rc.filter(col('Primary Type') == 'NARCOTICS').groupby(col("Description")).count().orderBy("count",ascending=False).show(1)

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|POSS: CANNABIS 30...| 1359|
+--------------------+-----+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import dayofweek

In [ ]:
rc.select(col("Date"),dayofweek(col("Date"))).show(5)

+-------------------+---------------+
|               Date|dayofweek(Date)|
+-------------------+---------------+
|2017-01-01 00:01:00|              1|
|2012-01-01 09:00:00|              1|
|2017-01-01 00:01:00|              1|
|2017-01-01 00:00:00|              1|
|2017-01-01 09:00:00|              1|
+-------------------+---------------+
only showing top 5 rows



# Using RDD

In [ ]:
psrdd=sc.textFile('/content/Police_Stations.csv')

psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [ ]:
ps_header=psrdd.first()

In [ ]:
ps_content=psrdd.filter(lambda line:line != ps_header)

ps_content.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

In [ ]:
ps_content.map(lambda line:line.split(',')).first()

['Headquarters',
 'Headquarters',
 '3510 S Michigan Ave',
 'Chicago',
 'IL',
 '60653',
 'http://home.chicagopolice.org',
 '',
 '',
 '',
 '1177731.401',
 '1881697.404',
 '41.83070169',
 '-87.62339535',
 '"(41.8307016873',
 ' -87.6233953459)"']

In [ ]:
ps_content.map(lambda line:line.split(',')).collect()

[['Headquarters',
  'Headquarters',
  '3510 S Michigan Ave',
  'Chicago',
  'IL',
  '60653',
  'http://home.chicagopolice.org',
  '',
  '',
  '',
  '1177731.401',
  '1881697.404',
  '41.83070169',
  '-87.62339535',
  '"(41.8307016873',
  ' -87.6233953459)"'],
 ['18',
  'Near North',
  '1160 N Larrabee St',
  'Chicago',
  'IL',
  '60610',
  'http://home.chicagopolice.org/community/districts/18th-district-near-north/',
  '312-742-5870',
  '312-742-5771',
  '312-742-5773',
  '1172080.029',
  '1908086.527',
  '41.90324165',
  '-87.64335214',
  '"(41.9032416531',
  ' -87.6433521393)"'],
 ['19',
  'Town Hall',
  '850 W Addison St',
  'Chicago',
  'IL',
  '60613',
  'http://home.chicagopolice.org/community/districts/19th-district-town-hall/',
  '312-744-8320',
  '312-744-4481',
  '312-744-8011',
  '1169730.744',
  '1924160.317',
  '41.94740046',
  '-87.65151202',
  '"(41.9474004564',
  ' -87.651512018)"'],
 ['20',
  'Lincoln',
  '5400 N Lincoln Ave',
  'Chicago',
  'IL',
  '60625',
  'http://

In [ ]:
ps_content.map(lambda line:line.split(',')).count()

24

In [ ]:
ps_content.filter(lambda line:line.split(',')[1]=='Englewood').map(lambda line:
    (line.split(',')[0],
     line.split(',')[2],
     line.split(',')[3],
     line.split(',')[4])).collect()

[('7', '1438 W 63rd St', 'Chicago', 'IL')]